In [1]:
# We import all our dependencies.
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.optimizer import Optimizer

import sys
sys.path.append('../')

from models.lvae import LadderVAE
from boilerplate import boilerplate
import lib.utils as utils
import training

import os
import glob
import zipfile
import urllib
from tifffile import imread, imsave
from matplotlib import pyplot as plt
from tqdm import tqdm

from pathlib import Path
from pystackreg import StackReg
from skimage.transform import warp,AffineTransform
import cv2

In [26]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [2]:
def crop_center(img,crop_size):

    if type(crop_size) == tuple:
        crop_x,crop_y = crop_size
    elif type(crop_size) == int:
        crop_x = crop_size
        crop_y = crop_size
    
    y,x = img.shape[-2::]
    startx = x//2-(crop_x//2)
    starty = y//2-(crop_y//2)        

    return img[...,starty:starty+crop_y,startx:startx+crop_x],startx,starty

In [49]:
initial_crop_size = 1200
infer_crop_size = 400



# Load imgs
gtpath = Path(r"E:\dl_monalisa\Data\Vim_fixed_mltplSNR_30nm\Evaluation\c14\gtavg\gtavg.tif")
HDNsuppath = Path(r"E:\dl_monalisa\Data\Vim_fixed_mltplSNR_30nm\Evaluation\c14\HDNsup003_1-2-3-4-0.tif")
HDNunsuppath = Path(r"E:\dl_monalisa\Data\Vim_fixed_mltplSNR_30nm\Evaluation\c14\HDNunsup_1-2-3-4-0_and0withunsupNoise0.tif")
CAREpath = Path(r"E:\dl_monalisa\Data\Vim_fixed_mltplSNR_30nm\Evaluation\c14\single_CAREpred.tiff")

gt = imread(gtpath)
predsHDNsup= imread(HDNsuppath)
predsHDNunsup= imread(HDNunsuppath)[...,0]
predCARE= imread(CAREpath)* np.std(gt) + np.mean(gt)



In [53]:
range_psnr = np.max(gt)-np.min(gt)

print("Single")
psnrCARE = utils.PSNR(gt, predCARE, range_psnr)
psnrHDNsup = utils.PSNR(gt, predsHDNsup[-1], range_psnr)
psnrHDNunsup1 = utils.PSNR(gt, predsHDNunsup[-2], range_psnr)
psnrHDNunsup2 = utils.PSNR(gt, predsHDNunsup[-1], range_psnr)
print(f"CARE: {psnrCARE}")
print(f"HDNsup: {psnrHDNsup}")
print(f"HDNunsup-1: {psnrHDNunsup1}")
print(f"HDNunsup-2: {psnrHDNunsup2}")

print("\nMltpl_snr\n")

for i in range(4):
    psnrHDNsup = utils.PSNR(gt, predsHDNsup[i], range_psnr)
    psnrHDNunsup = utils.PSNR(gt, predsHDNunsup[i], range_psnr)
    print(f"#{i} - sup: {psnrHDNsup} - unsup {psnrHDNunsup}")


Single
CARE: 30.689890384674072
HDNsup: 29.025437831878662
HDNunsup-1: 29.775114059448242
HDNunsup-2: 27.052288055419922

Mltpl_snr

#0 - sup: 29.355666637420654 - unsup 30.535452365875244
#1 - sup: 31.24502420425415 - unsup 30.888986587524414
#2 - sup: 28.885223865509033 - unsup 29.00089979171753
#3 - sup: 27.537050247192383 - unsup 25.773673057556152


In [42]:
from skimage.metrics import structural_similarity as ssim

ssimHDNsup = ssim(gt, predsHDNsup[-1], range = range_psnr)
ssimHDNunsup = ssim(gt, predsHDNunsup[-1], range = range_psnr)
ssimCARE = ssim(gt, predCARE, range = range_psnr)

print(ssimHDNsup)
print(ssimHDNunsup)
print(ssimCARE)

0.13179286829973566
0.18019237618989253
0.20160349477769463
